# Analyse de Fairness des Métadonnées de Radiographies Thoraciques

**Auteur:** Uthayakumar Tharushan  
**Date:** Janvier 2025  
**Projet:** Mi-parcours - Fairness en IA

## 1. Introduction (2/20)

### 1.1 Contexte et Cas d'usage

L'intelligence artificielle joue un rôle croissant dans le domaine médical, notamment pour l'analyse automatique d'images médicales. Les systèmes de diagnostic assisté par IA sont utilisés pour détecter des pathologies à partir de radiographies thoraciques, permettant ainsi d'améliorer la rapidité et la précision du diagnostic.

Cependant, ces systèmes peuvent hériter de biais présents dans les données d'entraînement, conduisant à des performances inégales selon les groupes démographiques (âge, genre). Par exemple, si un modèle est entraîné sur des données majoritairement composées d'hommes d'un certain âge, il pourrait moins bien performer pour les femmes ou les personnes plus jeunes/âgées.

### 1.2 Objectifs de l'Analyse

Ce projet vise à :
1. **Analyser** les métadonnées de 52,744 radiographies thoraciques pour identifier les biais démographiques
2. **Quantifier** les déséquilibres dans la distribution des patients (âge, genre) et des pathologies diagnostiquées
3. **Observer** les corrélations entre caractéristiques démographiques et diagnostics médicaux
4. **Appliquer** une méthode de pré-processing pour atténuer les biais identifiés
5. **Évaluer** l'efficacité de la méthode de mitigation

### 1.3 Importance de la Fairness en Santé

Dans le domaine médical, l'équité est cruciale car des prédictions biaisées peuvent avoir des conséquences graves sur la santé des patients. Un système diagnostique qui sous-performe pour certains groupes démographiques peut conduire à des diagnostics manqués ou erronés, aggravant les inégalités de santé existantes.

## 2. Préparation des Données (2/20)

### 2.1 Importation des Bibliothèques

In [1]:
# Manipulation de données
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning et Fairness
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from fairlearn.metrics import MetricFrame
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

# Statistiques
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu

# Configuration
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Bibliothèques importées avec succès !")

Bibliothèques importées avec succès !


### 2.2 Chargement des Données

Nous chargeons le fichier CSV contenant les métadonnées des radiographies thoraciques.

In [2]:
# Chargement du dataset
df = pd.read_csv('Uthayakumar_Tharushan.csv')

print(f"Nombre total d'observations : {len(df)}")
print(f"Nombre de colonnes : {len(df.columns)}")
print("\nAperçu des premières lignes :")
df.head()

Nombre total d'observations : 52742
Nombre de colonnes : 11

Aperçu des premières lignes :


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168,0.168
1,00000005_000.png,No Finding,0,5,69,F,PA,2048,2500,0.168,0.168
2,00000005_001.png,No Finding,1,5,69,F,AP,2500,2048,0.168,0.168
3,00000005_002.png,No Finding,2,5,69,F,AP,2500,2048,0.168,0.168
4,00000005_003.png,No Finding,3,5,69,F,PA,2992,2991,0.143,0.143


### 2.3 Exploration Initiale des Données

In [3]:
# Informations générales sur le dataset
print("=" * 80)
print("INFORMATIONS SUR LE DATASET")
print("=" * 80)
df.info()

print("\n" + "=" * 80)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 80)
df.describe(include='all')

INFORMATIONS SUR LE DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52742 entries, 0 to 52741
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  52742 non-null  object 
 1   Finding Labels               52742 non-null  object 
 2   Follow-up #                  52742 non-null  int64  
 3   Patient ID                   52742 non-null  int64  
 4   Patient Age                  52742 non-null  int64  
 5   Patient Gender               52742 non-null  object 
 6   View Position                52742 non-null  object 
 7   OriginalImage[Width          52742 non-null  int64  
 8   Height]                      52742 non-null  int64  
 9   OriginalImagePixelSpacing[x  52742 non-null  float64
 10  y]                           52742 non-null  float64
dtypes: float64(2), int64(5), object(4)
memory usage: 4.4+ MB

STATISTIQUES DESCRIPTIVES


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
count,52742,52742,52742.000000,52742.000000,52742.000000,52742,52742,52742.000000,52742.000000,52742.000000,52742.000000
unique,52742,621,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN
top,00000004_000.png,No Finding,NaN,NaN,NaN,M,PA,NaN,NaN,NaN,NaN
freq,1,28392,NaN,NaN,NaN,29857,32105,NaN,NaN,NaN,NaN
mean,NaN,NaN,7.858178,14310.728129,47.039172,NaN,NaN,2645.078988,2488.706249,0.155678,0.155678
std,NaN,NaN,13.814315,8344.733085,16.839064,NaN,NaN,342.355168,401.446349,0.016189,0.016189
min,NaN,NaN,0.000000,4.000000,1.000000,NaN,NaN,1215.000000,966.000000,0.115000,0.115000
25%,NaN,NaN,0.000000,7352.000000,35.000000,NaN,NaN,2500.000000,2048.000000,0.143000,0.143000
50%,NaN,NaN,3.000000,14022.000000,49.000000,NaN,NaN,2522.000000,2544.000000,0.143000,0.143000
75%,NaN,NaN,9.000000,20603.000000,59.000000,NaN,NaN,2992.000000,2991.000000,0.168000,0.168000


### 2.4 Gestion des Valeurs Manquantes

In [4]:
# Analyse des valeurs manquantes
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Colonne': missing_values.index,
    'Valeurs Manquantes': missing_values.values,
    'Pourcentage': missing_percent.values
})

missing_df = missing_df[missing_df['Valeurs Manquantes'] > 0].sort_values('Valeurs Manquantes', ascending=False)

if len(missing_df) > 0:
    print("Valeurs manquantes détectées :")
    print(missing_df)
    
    # Visualisation
    fig = px.bar(missing_df, x='Colonne', y='Pourcentage', 
                 title='Pourcentage de Valeurs Manquantes par Colonne',
                 labels={'Pourcentage': 'Pourcentage (%)', 'Colonne': 'Nom de la Colonne'})
    fig.show()
else:
    print("✓ Aucune valeur manquante détectée dans le dataset !")

✓ Aucune valeur manquante détectée dans le dataset !


### 2.5 Transformation et Nettoyage des Données

Nous procédons aux transformations nécessaires :
- Extraction des dimensions d'image
- Parsing des labels de pathologies (séparés par |)
- Gestion des valeurs aberrantes dans l'âge

In [5]:
# Création d'une copie pour les transformations
df_clean = df.copy()

# 1. Nettoyage de la colonne Age (gestion des valeurs aberrantes)
print("Distribution de l'âge avant nettoyage :")
print(df_clean['Patient Age'].describe())

# Suppression des âges aberrants (< 0 ou > 120)
age_avant = len(df_clean)
df_clean = df_clean[(df_clean['Patient Age'] >= 0) & (df_clean['Patient Age'] <= 120)]
age_apres = len(df_clean)

print(f"\n✓ {age_avant - age_apres} observations avec âge aberrant supprimées")
print(f"✓ {age_apres} observations conservées")

# 2. Extraction du nombre de pathologies
# Les labels sont séparés par '|', donc nous comptons le nombre de pathologies par patient
df_clean['Num_Findings'] = df_clean['Finding Labels'].apply(
    lambda x: 0 if pd.isna(x) or x == 'No Finding' else len(str(x).split('|'))
)

# 3. Indicateur de présence de pathologie
df_clean['Has_Finding'] = (df_clean['Finding Labels'] != 'No Finding') & (~df_clean['Finding Labels'].isna())
df_clean['Has_Finding'] = df_clean['Has_Finding'].astype(int)

# 4. Encodage du genre (M=1, F=0)
df_clean['Gender_Binary'] = (df_clean['Patient Gender'] == 'M').astype(int)

# 5. Catégorisation de l'âge en groupes
df_clean['Age_Group'] = pd.cut(df_clean['Patient Age'], 
                                bins=[0, 18, 40, 60, 120],
                                labels=['0-18', '19-40', '41-60', '60+'])

print("\n" + "=" * 80)
print("NOUVELLES COLONNES CRÉÉES")
print("=" * 80)
print("✓ Num_Findings : Nombre de pathologies par patient")
print("✓ Has_Finding : Indicateur binaire de présence de pathologie (1=oui, 0=non)")
print("✓ Gender_Binary : Genre encodé (1=M, 0=F)")
print("✓ Age_Group : Catégorisation de l'âge en 4 groupes")

print("\nAperçu des données nettoyées :")
df_clean.head()

Distribution de l'âge avant nettoyage :
count    52742.000000
mean        47.039172
std         16.839064
min          1.000000
25%         35.000000
50%         49.000000
75%         59.000000
max        412.000000
Name: Patient Age, dtype: float64

✓ 7 observations avec âge aberrant supprimées
✓ 52735 observations conservées

NOUVELLES COLONNES CRÉÉES
✓ Num_Findings : Nombre de pathologies par patient
✓ Has_Finding : Indicateur binaire de présence de pathologie (1=oui, 0=non)
✓ Gender_Binary : Genre encodé (1=M, 0=F)
✓ Age_Group : Catégorisation de l'âge en 4 groupes

Aperçu des données nettoyées :


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Num_Findings,Has_Finding,Gender_Binary,Age_Group
0,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168,0.168,2,1,1,60+
1,00000005_000.png,No Finding,0,5,69,F,PA,2048,2500,0.168,0.168,0,0,0,60+
2,00000005_001.png,No Finding,1,5,69,F,AP,2500,2048,0.168,0.168,0,0,0,60+
3,00000005_002.png,No Finding,2,5,69,F,AP,2500,2048,0.168,0.168,0,0,0,60+
4,00000005_003.png,No Finding,3,5,69,F,PA,2992,2991,0.143,0.143,0,0,0,60+


### 2.6 Extraction des Pathologies Individuelles

Nous créons des colonnes binaires pour chaque type de pathologie afin de faciliter l'analyse.

In [6]:
# Extraction de toutes les pathologies uniques
all_findings = []
for findings in df_clean['Finding Labels'].dropna():
    if findings != 'No Finding':
        all_findings.extend(str(findings).split('|'))

unique_findings = sorted(set(all_findings))
print(f"Nombre de pathologies uniques identifiées : {len(unique_findings)}")
print(f"\nListe des pathologies : {unique_findings}")

# Création de colonnes binaires pour chaque pathologie
for finding in unique_findings:
    df_clean[f'Has_{finding.replace(" ", "_")}'] = df_clean['Finding Labels'].apply(
        lambda x: 1 if pd.notna(x) and finding in str(x) else 0
    )

print(f"\n✓ {len(unique_findings)} colonnes binaires créées pour les pathologies")

Nombre de pathologies uniques identifiées : 14

Liste des pathologies : ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

✓ 14 colonnes binaires créées pour les pathologies


### 2.7 Résumé des Données Préparées

In [7]:
print("=" * 80)
print("RÉSUMÉ DES DONNÉES APRÈS PRÉPARATION")
print("=" * 80)
print(f"Nombre d'observations : {len(df_clean)}")
print(f"Nombre de colonnes : {len(df_clean.columns)}")
print(f"\nDistribution du genre :")
print(df_clean['Patient Gender'].value_counts())
print(f"\nDistribution des groupes d'âge :")
print(df_clean['Age_Group'].value_counts().sort_index())
print(f"\nPrésence de pathologies :")
print(df_clean['Has_Finding'].value_counts())

RÉSUMÉ DES DONNÉES APRÈS PRÉPARATION
Nombre d'observations : 52735
Nombre de colonnes : 29

Distribution du genre :
Patient Gender
M    29851
F    22884
Name: count, dtype: int64

Distribution des groupes d'âge :
Age_Group
0-18      2851
19-40    14613
41-60    23593
60+      11678
Name: count, dtype: int64

Présence de pathologies :
Has_Finding
0    28389
1    24346
Name: count, dtype: int64


## 3. Analyse Descriptive et Observation des Biais (7/20)

Cette section est la plus importante du projet. Nous allons analyser en détail les distributions et identifier les biais potentiels.

### 3.1 Analyse de la Distribution du Genre

In [8]:
# Distribution du genre
gender_counts = df_clean['Patient Gender'].value_counts()
gender_pct = df_clean['Patient Gender'].value_counts(normalize=True) * 100

print("=" * 80)
print("DISTRIBUTION DU GENRE")
print("=" * 80)
for gender in gender_counts.index:
    count = gender_counts[gender]
    pct = gender_pct[gender]
    print(f"{gender} : {count:,} patients ({pct:.2f}%)")

# Calcul du déséquilibre
ratio = gender_counts.max() / gender_counts.min()
print(f"\n⚠️ BIAIS DÉTECTÉ : Ratio de déséquilibre = {ratio:.2f}:1")
print(f"Le genre majoritaire représente {gender_pct.max():.2f}% du dataset")

# Visualisation
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type':'bar'}, {'type':'pie'}]],
    subplot_titles=('Distribution par Genre', 'Proportion par Genre')
)

fig.add_trace(
    go.Bar(x=gender_counts.index, y=gender_counts.values, 
           text=gender_counts.values, textposition='auto',
           marker_color=['lightblue', 'lightpink']),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=gender_counts.index, values=gender_counts.values,
           marker_colors=['lightblue', 'lightpink']),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, 
                  title_text="Analyse de la Distribution du Genre")
fig.show()

DISTRIBUTION DU GENRE
M : 29,851 patients (56.61%)
F : 22,884 patients (43.39%)

⚠️ BIAIS DÉTECTÉ : Ratio de déséquilibre = 1.30:1
Le genre majoritaire représente 56.61% du dataset


### 3.2 Analyse de la Distribution de l'Âge

In [9]:
# Statistiques descriptives de l'âge
print("=" * 80)
print("STATISTIQUES DE L'ÂGE")
print("=" * 80)
print(df_clean['Patient Age'].describe())

# Distribution par groupe d'âge
age_group_counts = df_clean['Age_Group'].value_counts().sort_index()
age_group_pct = (age_group_counts / len(df_clean)) * 100

print("\nDistribution par groupe d'âge :")
for group in age_group_counts.index:
    count = age_group_counts[group]
    pct = age_group_pct[group]
    print(f"{group} ans : {count:,} patients ({pct:.2f}%)")

# Identification du déséquilibre
age_ratio = age_group_counts.max() / age_group_counts.min()
print(f"\n⚠️ BIAIS DÉTECTÉ : Ratio de déséquilibre entre groupes d'âge = {age_ratio:.2f}:1")
print(f"Le groupe d'âge le plus représenté : {age_group_counts.idxmax()} ({age_group_pct.max():.2f}%)")
print(f"Le groupe d'âge le moins représenté : {age_group_counts.idxmin()} ({age_group_pct.min():.2f}%)")

STATISTIQUES DE L'ÂGE
count    52735.000000
mean        47.015436
std         16.658820
min          1.000000
25%         35.000000
50%         49.000000
75%         59.000000
max         95.000000
Name: Patient Age, dtype: float64

Distribution par groupe d'âge :
0-18 ans : 2,851 patients (5.41%)
19-40 ans : 14,613 patients (27.71%)
41-60 ans : 23,593 patients (44.74%)
60+ ans : 11,678 patients (22.14%)

⚠️ BIAIS DÉTECTÉ : Ratio de déséquilibre entre groupes d'âge = 8.28:1
Le groupe d'âge le plus représenté : 41-60 (44.74%)
Le groupe d'âge le moins représenté : 0-18 (5.41%)


In [10]:
# Visualisations de la distribution de l'âge
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribution Continue de l\'Âge', 
                    'Distribution par Groupe d\'Âge',
                    'Boxplot de l\'Âge par Genre',
                    'Distribution de l\'Âge par Genre'),
    specs=[[{'type':'histogram'}, {'type':'bar'}],
           [{'type':'box'}, {'type':'violin'}]]
)

# Histogramme de l'âge
fig.add_trace(
    go.Histogram(x=df_clean['Patient Age'], nbinsx=50, name='Âge',
                 marker_color='skyblue'),
    row=1, col=1
)

# Bar chart par groupe d'âge
fig.add_trace(
    go.Bar(x=age_group_counts.index, y=age_group_counts.values,
           text=age_group_counts.values, textposition='auto',
           marker_color='lightcoral'),
    row=1, col=2
)

# Boxplot par genre
for gender in df_clean['Patient Gender'].unique():
    data = df_clean[df_clean['Patient Gender'] == gender]['Patient Age']
    fig.add_trace(
        go.Box(y=data, name=gender, marker_color='lightblue' if gender == 'M' else 'lightpink'),
        row=2, col=1
    )

# Violin plot par genre
for gender in df_clean['Patient Gender'].unique():
    data = df_clean[df_clean['Patient Gender'] == gender]['Patient Age']
    fig.add_trace(
        go.Violin(y=data, name=gender, marker_color='lightblue' if gender == 'M' else 'lightpink'),
        row=2, col=2
    )

fig.update_layout(height=800, showlegend=True, 
                  title_text="Analyse Complète de la Distribution de l'Âge")
fig.show()

### 3.3 Analyse Croisée : Genre et Âge

In [11]:
# Test statistique : différence d'âge entre genres
age_male = df_clean[df_clean['Patient Gender'] == 'M']['Patient Age']
age_female = df_clean[df_clean['Patient Gender'] == 'F']['Patient Age']

# Test de Mann-Whitney U
statistic, p_value = mannwhitneyu(age_male, age_female)

print("=" * 80)
print("ANALYSE CROISÉE : GENRE ET ÂGE")
print("=" * 80)
print(f"Âge moyen (Hommes) : {age_male.mean():.2f} ans (σ = {age_male.std():.2f})")
print(f"Âge moyen (Femmes) : {age_female.mean():.2f} ans (σ = {age_female.std():.2f})")
print(f"\nTest de Mann-Whitney U : p-value = {p_value:.4e}")

if p_value < 0.05:
    print("⚠️ BIAIS DÉTECTÉ : Différence significative d'âge entre les genres (p < 0.05)")
else:
    print("✓ Pas de différence significative d'âge entre les genres")

# Tableau croisé : Genre × Groupe d'âge
cross_tab = pd.crosstab(df_clean['Age_Group'], df_clean['Patient Gender'], margins=True)
print("\nTableau croisé Genre × Groupe d'Âge :")
print(cross_tab)

# Pourcentages
cross_tab_pct = pd.crosstab(df_clean['Age_Group'], df_clean['Patient Gender'], normalize='columns') * 100
print("\nPourcentages par colonne :")
print(cross_tab_pct.round(2))

ANALYSE CROISÉE : GENRE ET ÂGE
Âge moyen (Hommes) : 47.35 ans (σ = 16.98)
Âge moyen (Femmes) : 46.58 ans (σ = 16.22)

Test de Mann-Whitney U : p-value = 3.4322e-12
⚠️ BIAIS DÉTECTÉ : Différence significative d'âge entre les genres (p < 0.05)

Tableau croisé Genre × Groupe d'Âge :
Patient Gender      F      M    All
Age_Group                          
0-18             1201   1650   2851
19-40            6541   8072  14613
41-60           10589  13004  23593
60+              4553   7125  11678
All             22884  29851  52735

Pourcentages par colonne :
Patient Gender      F      M
Age_Group                   
0-18             5.25   5.53
19-40           28.58  27.04
41-60           46.27  43.56
60+             19.90  23.87


### 3.4 Analyse des Pathologies

Analyse de la distribution des pathologies diagnostiquées.

In [12]:
# Distribution des pathologies
print("=" * 80)
print("DISTRIBUTION DES PATHOLOGIES")
print("=" * 80)

# Patients avec/sans pathologie
has_finding_counts = df_clean['Has_Finding'].value_counts()
has_finding_pct = (has_finding_counts / len(df_clean)) * 100

print(f"Patients sans pathologie : {has_finding_counts[0]:,} ({has_finding_pct[0]:.2f}%)")
print(f"Patients avec pathologie(s) : {has_finding_counts[1]:,} ({has_finding_pct[1]:.2f}%)")

# Comptage de chaque pathologie
findings_counts = {}
for finding in unique_findings:
    col_name = f'Has_{finding.replace(" ", "_")}'
    findings_counts[finding] = df_clean[col_name].sum()

findings_df = pd.DataFrame(list(findings_counts.items()), columns=['Pathologie', 'Nombre'])
findings_df = findings_df.sort_values('Nombre', ascending=False)
findings_df['Pourcentage'] = (findings_df['Nombre'] / len(df_clean)) * 100

print("\nDistribution détaillée des pathologies :")
print(findings_df.to_string(index=False))

# Identification du déséquilibre
if len(findings_df) > 0:
    ratio_findings = findings_df['Nombre'].max() / findings_df['Nombre'].min()
    print(f"\n⚠️ BIAIS DÉTECTÉ : Ratio de déséquilibre entre pathologies = {ratio_findings:.2f}:1")
    print(f"Pathologie la plus fréquente : {findings_df.iloc[0]['Pathologie']} ({findings_df.iloc[0]['Pourcentage']:.2f}%)")
    print(f"Pathologie la moins fréquente : {findings_df.iloc[-1]['Pathologie']} ({findings_df.iloc[-1]['Pourcentage']:.2f}%)")

DISTRIBUTION DES PATHOLOGIES
Patients sans pathologie : 28,389 (53.83%)
Patients avec pathologie(s) : 24,346 (46.17%)

Distribution détaillée des pathologies :
        Pathologie  Nombre  Pourcentage
      Infiltration    9336    17.703612
          Effusion    6208    11.772068
       Atelectasis    5583    10.586897
            Nodule    2962     5.616763
              Mass    2712     5.142695
      Pneumothorax    2423     4.594671
     Consolidation    2163     4.101640
Pleural_Thickening    1610     3.053001
      Cardiomegaly    1256     2.381720
         Emphysema    1229     2.330521
             Edema    1050     1.991088
          Fibrosis     898     1.702854
         Pneumonia     718     1.361525
            Hernia     105     0.199109

⚠️ BIAIS DÉTECTÉ : Ratio de déséquilibre entre pathologies = 88.91:1
Pathologie la plus fréquente : Infiltration (17.70%)
Pathologie la moins fréquente : Hernia (0.20%)


In [13]:
# Visualisation des pathologies
fig = px.bar(findings_df.head(15), x='Pathologie', y='Nombre',
             title='Top 15 des Pathologies les Plus Fréquentes',
             labels={'Nombre': 'Nombre de Cas', 'Pathologie': 'Type de Pathologie'},
             text='Nombre')
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

# Distribution du nombre de pathologies par patient
num_findings_dist = df_clean['Num_Findings'].value_counts().sort_index()

fig = px.bar(x=num_findings_dist.index, y=num_findings_dist.values,
             title='Distribution du Nombre de Pathologies par Patient',
             labels={'x': 'Nombre de Pathologies', 'y': 'Nombre de Patients'},
             text=num_findings_dist.values)
fig.update_traces(textposition='outside')
fig.show()

print(f"Nombre moyen de pathologies par patient : {df_clean['Num_Findings'].mean():.2f}")
print(f"Nombre maximum de pathologies pour un patient : {df_clean['Num_Findings'].max()}")

Nombre moyen de pathologies par patient : 0.73
Nombre maximum de pathologies pour un patient : 9


### 3.5 Analyse Croisée : Genre et Pathologies

**Analyse critique pour identifier les biais de genre dans les diagnostics.**

In [14]:
print("=" * 80)
print("ANALYSE CROISÉE : GENRE ET PATHOLOGIES")
print("=" * 80)

# Taux de présence de pathologie par genre
findings_by_gender = df_clean.groupby('Patient Gender')['Has_Finding'].agg(['sum', 'count', 'mean'])
findings_by_gender.columns = ['Avec Pathologie', 'Total', 'Taux']
findings_by_gender['Taux (%)'] = findings_by_gender['Taux'] * 100

print("\nTaux de présence de pathologie par genre :")
print(findings_by_gender)

# Test de chi-2 pour l'indépendance
contingency_table = pd.crosstab(df_clean['Patient Gender'], df_clean['Has_Finding'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"\nTest du Chi-2 d'indépendance :")
print(f"Chi-2 = {chi2:.4f}, p-value = {p_value:.4e}")

if p_value < 0.05:
    print("⚠️ BIAIS DÉTECTÉ : Le genre et la présence de pathologie ne sont PAS indépendants (p < 0.05)")
    print("   Cela suggère un biais potentiel dans la distribution des diagnostics selon le genre.")
else:
    print("✓ Le genre et la présence de pathologie semblent indépendants")

# Analyse détaillée par pathologie
print("\n" + "=" * 80)
print("TAUX DE PRÉVALENCE PAR PATHOLOGIE ET GENRE")
print("=" * 80)

pathology_gender_analysis = []
for finding in unique_findings[:10]:  # Top 10 pathologies
    col_name = f'Has_{finding.replace(" ", "_")}'
    
    rate_m = df_clean[df_clean['Patient Gender'] == 'M'][col_name].mean() * 100
    rate_f = df_clean[df_clean['Patient Gender'] == 'F'][col_name].mean() * 100
    
    pathology_gender_analysis.append({
        'Pathologie': finding,
        'Taux Hommes (%)': rate_m,
        'Taux Femmes (%)': rate_f,
        'Différence': abs(rate_m - rate_f),
        'Ratio (M/F)': rate_m / rate_f if rate_f > 0 else np.inf
    })

pathology_gender_df = pd.DataFrame(pathology_gender_analysis)
pathology_gender_df = pathology_gender_df.sort_values('Différence', ascending=False)

print(pathology_gender_df.to_string(index=False))

# Identification des biais les plus importants
significant_bias = pathology_gender_df[pathology_gender_df['Différence'] > 2.0]
if len(significant_bias) > 0:
    print(f"\n⚠️ BIAIS SIGNIFICATIFS DÉTECTÉS ({len(significant_bias)} pathologies avec >2% de différence) :")
    for _, row in significant_bias.iterrows():
        higher_gender = 'Hommes' if row['Taux Hommes (%)'] > row['Taux Femmes (%)'] else 'Femmes'
        print(f"   - {row['Pathologie']} : {row['Différence']:.2f}% de différence (plus fréquent chez les {higher_gender})")

ANALYSE CROISÉE : GENRE ET PATHOLOGIES

Taux de présence de pathologie par genre :
                Avec Pathologie  Total      Taux   Taux (%)
Patient Gender                                             
F                         10428  22884  0.455690  45.568956
M                         13918  29851  0.466249  46.624904

Test du Chi-2 d'indépendance :
Chi-2 = 5.7692, p-value = 1.6309e-02
⚠️ BIAIS DÉTECTÉ : Le genre et la présence de pathologie ne sont PAS indépendants (p < 0.05)
   Cela suggère un biais potentiel dans la distribution des diagnostics selon le genre.

TAUX DE PRÉVALENCE PAR PATHOLOGIE ET GENRE
   Pathologie  Taux Hommes (%)  Taux Femmes (%)  Différence  Ratio (M/F)
  Atelectasis        11.366453         9.570005    1.796448     1.187717
         Mass         5.725101         4.382975    1.342126     1.306214
 Cardiomegaly         1.953033         2.940919    0.987886     0.664089
     Effusion        11.379853        12.283692    0.903838     0.926420
    Emphysema     

In [15]:
# Visualisation comparative par genre
top_findings = findings_df.head(10)['Pathologie'].tolist()

gender_comparison_data = []
for finding in top_findings:
    col_name = f'Has_{finding.replace(" ", "_")}'
    for gender in ['M', 'F']:
        rate = df_clean[df_clean['Patient Gender'] == gender][col_name].mean() * 100
        gender_comparison_data.append({
            'Pathologie': finding,
            'Genre': gender,
            'Taux (%)': rate
        })

gender_comparison_df = pd.DataFrame(gender_comparison_data)

fig = px.bar(gender_comparison_df, x='Pathologie', y='Taux (%)', color='Genre',
             title='Comparaison des Taux de Prévalence par Pathologie et Genre (Top 10)',
             barmode='group',
             color_discrete_map={'M': 'lightblue', 'F': 'lightpink'})
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

### 3.6 Analyse Croisée : Âge et Pathologies

In [16]:
print("=" * 80)
print("ANALYSE CROISÉE : ÂGE ET PATHOLOGIES")
print("=" * 80)

# Taux de présence de pathologie par groupe d'âge
findings_by_age = df_clean.groupby('Age_Group')['Has_Finding'].agg(['sum', 'count', 'mean'])
findings_by_age.columns = ['Avec Pathologie', 'Total', 'Taux']
findings_by_age['Taux (%)'] = findings_by_age['Taux'] * 100

print("\nTaux de présence de pathologie par groupe d'âge :")
print(findings_by_age)

# Test de chi-2
contingency_age = pd.crosstab(df_clean['Age_Group'], df_clean['Has_Finding'])
chi2_age, p_value_age, dof_age, expected_age = chi2_contingency(contingency_age)

print(f"\nTest du Chi-2 d'indépendance (Âge × Pathologie) :")
print(f"Chi-2 = {chi2_age:.4f}, p-value = {p_value_age:.4e}")

if p_value_age < 0.05:
    print("⚠️ BIAIS DÉTECTÉ : L'âge et la présence de pathologie ne sont PAS indépendants (p < 0.05)")
    print("   Les personnes plus âgées ont tendance à avoir plus de pathologies.")
else:
    print("✓ L'âge et la présence de pathologie semblent indépendants")

# Nombre moyen de pathologies par groupe d'âge
avg_findings_by_age = df_clean.groupby('Age_Group')['Num_Findings'].mean()
print("\nNombre moyen de pathologies par groupe d'âge :")
print(avg_findings_by_age)

ANALYSE CROISÉE : ÂGE ET PATHOLOGIES

Taux de présence de pathologie par groupe d'âge :
           Avec Pathologie  Total      Taux   Taux (%)
Age_Group                                             
0-18                  1227   2851  0.430375  43.037531
19-40                 5995  14613  0.410251  41.025115
41-60                11015  23593  0.466876  46.687577
60+                   6109  11678  0.523120  52.312040

Test du Chi-2 d'indépendance (Âge × Pathologie) :
Chi-2 = 346.6966, p-value = 7.7437e-75
⚠️ BIAIS DÉTECTÉ : L'âge et la présence de pathologie ne sont PAS indépendants (p < 0.05)
   Les personnes plus âgées ont tendance à avoir plus de pathologies.

Nombre moyen de pathologies par groupe d'âge :
Age_Group
0-18     0.672746
19-40    0.636488
41-60    0.731573
60+      0.836958
Name: Num_Findings, dtype: float64


In [17]:
# Visualisation
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Taux de Présence de Pathologie par Âge', 
                    'Nombre Moyen de Pathologies par Âge')
)

fig.add_trace(
    go.Bar(x=findings_by_age.index, y=findings_by_age['Taux (%)'],
           text=findings_by_age['Taux (%)'].round(2), textposition='auto',
           marker_color='lightcoral'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=avg_findings_by_age.index, y=avg_findings_by_age.values,
           text=avg_findings_by_age.values.round(2), textposition='auto',
           marker_color='lightskyblue'),
    row=1, col=2
)

fig.update_xaxes(title_text="Groupe d'Âge", row=1, col=1)
fig.update_xaxes(title_text="Groupe d'Âge", row=1, col=2)
fig.update_yaxes(title_text="Taux (%)", row=1, col=1)
fig.update_yaxes(title_text="Nombre Moyen", row=1, col=2)

fig.update_layout(height=400, showlegend=False, 
                  title_text="Analyse de la Relation Âge-Pathologies")
fig.show()

### 3.7 Synthèse des Biais Identifiés

In [18]:
print("=" * 80)
print("SYNTHÈSE DES BIAIS IDENTIFIÉS DANS LE DATASET")
print("=" * 80)

bias_summary = []

# Biais de genre
gender_ratio = gender_counts.max() / gender_counts.min()
bias_summary.append({
    'Type de Biais': 'Déséquilibre de Genre',
    'Métrique': f'Ratio {gender_ratio:.2f}:1',
    'Sévérité': 'ÉLEVÉE' if gender_ratio > 2 else 'MODÉRÉE' if gender_ratio > 1.5 else 'FAIBLE',
    'Description': f"Le genre {gender_counts.idxmax()} est sur-représenté ({gender_pct.max():.1f}%)"
})

# Biais d'âge
age_ratio = age_group_counts.max() / age_group_counts.min()
bias_summary.append({
    'Type de Biais': "Déséquilibre d'Âge",
    'Métrique': f'Ratio {age_ratio:.2f}:1',
    'Sévérité': 'ÉLEVÉE' if age_ratio > 3 else 'MODÉRÉE' if age_ratio > 2 else 'FAIBLE',
    'Description': f"Le groupe {age_group_counts.idxmax()} est sur-représenté ({age_group_pct.max():.1f}%)"
})

# Biais dans les pathologies
if len(findings_df) > 0:
    pathology_ratio = findings_df['Nombre'].max() / findings_df['Nombre'].min()
    bias_summary.append({
        'Type de Biais': 'Déséquilibre des Pathologies',
        'Métrique': f'Ratio {pathology_ratio:.2f}:1',
        'Sévérité': 'ÉLEVÉE' if pathology_ratio > 10 else 'MODÉRÉE' if pathology_ratio > 5 else 'FAIBLE',
        'Description': f"La pathologie '{findings_df.iloc[0]['Pathologie']}' est sur-représentée"
    })

# Biais d'interaction genre-pathologie
if p_value < 0.05:
    bias_summary.append({
        'Type de Biais': 'Interaction Genre-Pathologie',
        'Métrique': f'p-value = {p_value:.4e}',
        'Sévérité': 'ÉLEVÉE',
        'Description': "Dépendance statistique significative entre genre et présence de pathologie"
    })

# Biais d'interaction âge-pathologie
if p_value_age < 0.05:
    bias_summary.append({
        'Type de Biais': 'Interaction Âge-Pathologie',
        'Métrique': f'p-value = {p_value_age:.4e}',
        'Sévérité': 'ÉLEVÉE',
        'Description': "Dépendance statistique significative entre âge et présence de pathologie"
    })

bias_df = pd.DataFrame(bias_summary)
print("\n", bias_df.to_string(index=False))

print("\n" + "=" * 80)
print("IMPLICATIONS POUR L'ÉQUITÉ DES MODÈLES")
print("=" * 80)
print("""
Les biais identifiés peuvent avoir plusieurs conséquences :

1. BIAIS DE REPRÉSENTATION : Les groupes sous-représentés (genre minoritaire, 
   certains groupes d'âge) risquent d'être moins bien modélisés.

2. BIAIS DE PRÉDICTION : Un modèle entraîné sur ces données pourrait avoir des
   performances inégales selon le genre ou l'âge du patient.

3. BIAIS DE DIAGNOSTIC : Les corrélations genre/âge-pathologie peuvent mener à
   des sur-diagnostics ou sous-diagnostics pour certains groupes.

4. NÉCESSITÉ DE MITIGATION : Il est crucial d'appliquer des techniques de
   pré-processing pour équilibrer le dataset avant l'entraînement de modèles.
""")

SYNTHÈSE DES BIAIS IDENTIFIÉS DANS LE DATASET

                Type de Biais             Métrique Sévérité                                                                Description
       Déséquilibre de Genre         Ratio 1.30:1   FAIBLE                                      Le genre M est sur-représenté (56.6%)
          Déséquilibre d'Âge         Ratio 8.28:1   ÉLEVÉE                                 Le groupe 41-60 est sur-représenté (44.7%)
Déséquilibre des Pathologies        Ratio 88.91:1   ÉLEVÉE                           La pathologie 'Infiltration' est sur-représentée
Interaction Genre-Pathologie p-value = 1.6309e-02   ÉLEVÉE Dépendance statistique significative entre genre et présence de pathologie
  Interaction Âge-Pathologie p-value = 7.7437e-75   ÉLEVÉE   Dépendance statistique significative entre âge et présence de pathologie

IMPLICATIONS POUR L'ÉQUITÉ DES MODÈLES

Les biais identifiés peuvent avoir plusieurs conséquences :

1. BIAIS DE REPRÉSENTATION : Les groupes sous-

## 4. Méthode de Mitigation des Biais par Pré-processing (3/20)

Nous allons appliquer une technique de **reweighting (repondération)** pour atténuer les biais identifiés. Cette méthode attribue des poids aux observations pour équilibrer la représentation des différents groupes.

### 4.1 Préparation pour la Mitigation

In [19]:
# Création d'un attribut sensible combiné (Genre + Groupe d'âge)
df_clean['Sensitive_Attribute'] = df_clean['Patient Gender'] + '_' + df_clean['Age_Group'].astype(str)

print("=" * 80)
print("PRÉPARATION POUR LA MITIGATION")
print("=" * 80)

# Distribution de l'attribut sensible
sensitive_dist = df_clean['Sensitive_Attribute'].value_counts().sort_index()
print("\nDistribution de l'attribut sensible (Genre × Âge) :")
print(sensitive_dist)

print(f"\nNombre de sous-groupes : {len(sensitive_dist)}")
print(f"Groupe le plus représenté : {sensitive_dist.idxmax()} ({sensitive_dist.max()} observations)")
print(f"Groupe le moins représenté : {sensitive_dist.idxmin()} ({sensitive_dist.min()} observations)")
print(f"Ratio max/min : {sensitive_dist.max() / sensitive_dist.min():.2f}:1")

PRÉPARATION POUR LA MITIGATION

Distribution de l'attribut sensible (Genre × Âge) :
Sensitive_Attribute
F_0-18      1201
F_19-40     6541
F_41-60    10589
F_60+       4553
M_0-18      1650
M_19-40     8072
M_41-60    13004
M_60+       7125
Name: count, dtype: int64

Nombre de sous-groupes : 8
Groupe le plus représenté : M_41-60 (13004 observations)
Groupe le moins représenté : F_0-18 (1201 observations)
Ratio max/min : 10.83:1


### 4.2 Calcul des Poids de Repondération

Nous calculons des poids pour chaque observation de manière à équilibrer la représentation de chaque sous-groupe (combinaison genre × âge).

In [20]:
# Calcul des poids inversement proportionnels à la fréquence du groupe
# Poids = 1 / (fréquence du groupe)

group_counts = df_clean['Sensitive_Attribute'].value_counts()
group_weights = 1.0 / group_counts

# Normalisation des poids pour que leur somme = nombre d'observations
group_weights = group_weights * (len(df_clean) / group_weights.sum())

# Attribution des poids à chaque observation
df_clean['Sample_Weight'] = df_clean['Sensitive_Attribute'].map(group_weights)

print("=" * 80)
print("CALCUL DES POIDS DE REPONDÉRATION")
print("=" * 80)

print("\nPoids par sous-groupe :")
weight_summary = pd.DataFrame({
    'Groupe': group_weights.index,
    'Effectif': group_counts[group_weights.index].values,
    'Poids': group_weights.values
})
weight_summary = weight_summary.sort_values('Effectif', ascending=False)
print(weight_summary.to_string(index=False))

print(f"\nStatistiques des poids :")
print(f"  Poids minimum : {df_clean['Sample_Weight'].min():.4f}")
print(f"  Poids maximum : {df_clean['Sample_Weight'].max():.4f}")
print(f"  Poids moyen : {df_clean['Sample_Weight'].mean():.4f}")
print(f"  Ratio max/min : {df_clean['Sample_Weight'].max() / df_clean['Sample_Weight'].min():.2f}:1")

print("\n✓ Les groupes sous-représentés reçoivent des poids plus élevés.")
print("✓ Les groupes sur-représentés reçoivent des poids plus faibles.")
print("✓ Cela équilibre l'importance de chaque sous-groupe dans l'entraînement des modèles.")

CALCUL DES POIDS DE REPONDÉRATION

Poids par sous-groupe :
 Groupe  Effectif        Poids
M_41-60     13004  1804.926239
F_41-60     10589  2216.570102
M_19-40      8072  2907.737960
  M_60+      7125  3294.212044
F_19-40      6541  3588.329126
  F_60+      4553  5155.119880
 M_0-18      1650 14225.006553
 F_0-18      1201 19543.098096

Statistiques des poids :
  Poids minimum : 1804.9262
  Poids maximum : 19543.0981
  Poids moyen : 3560.6350
  Ratio max/min : 10.83:1

✓ Les groupes sous-représentés reçoivent des poids plus élevés.
✓ Les groupes sur-représentés reçoivent des poids plus faibles.
✓ Cela équilibre l'importance de chaque sous-groupe dans l'entraînement des modèles.


### 4.3 Visualisation de l'Effet de la Repondération

In [21]:
# Comparaison avant/après repondération
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribution Originale (Genre)', 
                    'Distribution Pondérée (Genre)')
)

# Distribution originale
original_counts = df_clean['Patient Gender'].value_counts()
fig.add_trace(
    go.Bar(x=original_counts.index, y=original_counts.values,
           text=original_counts.values, textposition='auto',
           marker_color=['lightblue', 'lightpink'], name='Original'),
    row=1, col=1
)

# Distribution pondérée (somme des poids par groupe)
weighted_counts = df_clean.groupby('Patient Gender')['Sample_Weight'].sum()
fig.add_trace(
    go.Bar(x=weighted_counts.index, y=weighted_counts.values,
           text=[f"{v:.0f}" for v in weighted_counts.values], textposition='auto',
           marker_color=['lightblue', 'lightpink'], name='Pondéré'),
    row=1, col=2
)

fig.update_xaxes(title_text="Genre", row=1, col=1)
fig.update_xaxes(title_text="Genre", row=1, col=2)
fig.update_yaxes(title_text="Effectif", row=1, col=1)
fig.update_yaxes(title_text="Poids Total", row=1, col=2)

fig.update_layout(height=400, showlegend=False, 
                  title_text="Effet de la Repondération sur la Distribution du Genre")
fig.show()

# Calcul de l'amélioration
original_ratio = original_counts.max() / original_counts.min()
weighted_ratio = weighted_counts.max() / weighted_counts.min()

print("\n" + "=" * 80)
print("ÉVALUATION DE LA MITIGATION")
print("=" * 80)
print(f"Ratio de déséquilibre AVANT repondération : {original_ratio:.2f}:1")
print(f"Ratio de déséquilibre APRÈS repondération : {weighted_ratio:.2f}:1")
improvement = ((original_ratio - weighted_ratio) / original_ratio) * 100
print(f"\n✓ Amélioration du déséquilibre : {improvement:.2f}%")


ÉVALUATION DE LA MITIGATION
Ratio de déséquilibre AVANT repondération : 1.30:1
Ratio de déséquilibre APRÈS repondération : 1.00:1

✓ Amélioration du déséquilibre : 23.34%


### 4.4 Sauvegarde du Dataset Transformé

In [22]:
# Sélection des colonnes pertinentes pour le dataset final
columns_to_save = [
    'Image Index', 'Patient ID', 'Patient Age', 'Patient Gender', 'Age_Group',
    'Finding Labels', 'Has_Finding', 'Num_Findings', 'View Position',
    'Sensitive_Attribute', 'Sample_Weight'
]

# Ajout des colonnes de pathologies
pathology_columns = [col for col in df_clean.columns if col.startswith('Has_')]
columns_to_save.extend(pathology_columns)

# Création du dataset final
df_mitigated = df_clean[columns_to_save].copy()

# Sauvegarde
output_file = 'Dataset_Mitige_Tharushan.csv'
df_mitigated.to_csv(output_file, index=False)

print("=" * 80)
print("DATASET TRANSFORMÉ SAUVEGARDÉ")
print("=" * 80)
print(f"Fichier : {output_file}")
print(f"Nombre d'observations : {len(df_mitigated):,}")
print(f"Nombre de colonnes : {len(df_mitigated.columns)}")
print("\n✓ Le dataset inclut la colonne 'Sample_Weight' pour l'entraînement équitable des modèles.")
print("✓ Les colonnes binaires pour chaque pathologie sont incluses.")
print("✓ L'attribut sensible combiné (Genre × Âge) est inclus pour l'analyse de fairness.")

print("\nAperçu du dataset final :")
df_mitigated.head(10)

DATASET TRANSFORMÉ SAUVEGARDÉ
Fichier : Dataset_Mitige_Tharushan.csv
Nombre d'observations : 52,735
Nombre de colonnes : 26

✓ Le dataset inclut la colonne 'Sample_Weight' pour l'entraînement équitable des modèles.
✓ Les colonnes binaires pour chaque pathologie sont incluses.
✓ L'attribut sensible combiné (Genre × Âge) est inclus pour l'analyse de fairness.

Aperçu du dataset final :


,Image Index,Patient ID,Patient Age,Patient Gender,Age_Group,Finding Labels,Has_Finding,Num_Findings,View Position,Sensitive_Attribute,...,Has_Effusion,Has_Emphysema,Has_Fibrosis,Has_Hernia,Has_Infiltration,Has_Mass,Has_Nodule,Has_Pleural_Thickening,Has_Pneumonia,Has_Pneumothorax
0,00000004_000.png,4,82,M,60+,Mass|Nodule,1,2,AP,M_60+,...,0,0,0,0,0,1,1,0,0,0
1,00000005_000.png,5,69,F,60+,No Finding,0,0,PA,F_60+,...,0,0,0,0,0,0,0,0,0,0
2,00000005_001.png,5,69,F,60+,No Finding,0,0,AP,F_60+,...,0,0,0,0,0,0,0,0,0,0
3,00000005_002.png,5,69,F,60+,No Finding,0,0,AP,F_60+,...,0,0,0,0,0,0,0,0,0,0
4,00000005_003.png,5,69,F,60+,No Finding,0,0,PA,F_60+,...,0,0,0,0,0,0,0,0,0,0
5,00000005_004.png,5,70,F,60+,No Finding,0,0,PA,F_60+,...,0,0,0,0,0,0,0,0,0,0
6,00000005_005.png,5,70,F,60+,No Finding,0,0,PA,F_60+,...,0,0,0,0,0,0,0,0,0,0
7,00000005_006.png,5,70,F,60+,Infiltration,1,1,PA,F_60+,...,0,0,0,0,1,0,0,0,0,0
8,00000005_007.png,5,70,F,60+,Effusion|Infiltration,1,2,PA,F_60+,...,1,0,0,0,1,0,0,0,0,0
9,00000008_000.png,8,69,F,60+,Cardiomegaly,1,1,PA,F_60+,...,0,0,0,0,0,0,0,0,0,0


### 4.5 Métriques de Fairness Avant/Après Mitigation

Nous évaluons l'impact de la repondération sur les métriques de fairness.

In [23]:
print("=" * 80)
print("MÉTRIQUES DE FAIRNESS")
print("=" * 80)

# Calcul du taux de positivité (présence de pathologie) par genre
print("\n1. STATISTICAL PARITY (Parité Statistique)")
print("   Mesure : Égalité des taux de prédiction positive entre groupes\n")

# Avant mitigation (taux non pondéré)
positive_rate_by_gender = df_clean.groupby('Patient Gender')['Has_Finding'].mean()
print("   AVANT mitigation (taux non pondérés) :")
for gender, rate in positive_rate_by_gender.items():
    print(f"      {gender} : {rate*100:.2f}%")

parity_diff_before = abs(positive_rate_by_gender.max() - positive_rate_by_gender.min())
print(f"   Différence : {parity_diff_before*100:.2f}%")

# Après mitigation (taux pondéré)
weighted_positive_rate = {}
for gender in df_clean['Patient Gender'].unique():
    mask = df_clean['Patient Gender'] == gender
    weighted_rate = np.average(
        df_clean.loc[mask, 'Has_Finding'],
        weights=df_clean.loc[mask, 'Sample_Weight']
    )
    weighted_positive_rate[gender] = weighted_rate

print("\n   APRÈS mitigation (taux pondérés) :")
for gender, rate in weighted_positive_rate.items():
    print(f"      {gender} : {rate*100:.2f}%")

parity_diff_after = abs(max(weighted_positive_rate.values()) - min(weighted_positive_rate.values()))
print(f"   Différence : {parity_diff_after*100:.2f}%")

print(f"\n   ✓ Amélioration de la parité statistique : {((parity_diff_before - parity_diff_after) / parity_diff_before * 100):.2f}%")

# Analyse par groupe d'âge
print("\n" + "="*80)
print("2. FAIRNESS PAR GROUPE D'ÂGE")
print("="*80)

positive_rate_by_age = df_clean.groupby('Age_Group')['Has_Finding'].mean()
print("\n   AVANT mitigation (taux non pondérés) :")
for age, rate in positive_rate_by_age.items():
    print(f"      {age} : {rate*100:.2f}%")

age_parity_diff_before = abs(positive_rate_by_age.max() - positive_rate_by_age.min())
print(f"   Différence max : {age_parity_diff_before*100:.2f}%")

weighted_positive_rate_age = {}
for age in df_clean['Age_Group'].cat.categories:
    mask = df_clean['Age_Group'] == age
    if mask.sum() > 0:
        weighted_rate = np.average(
            df_clean.loc[mask, 'Has_Finding'],
            weights=df_clean.loc[mask, 'Sample_Weight']
        )
        weighted_positive_rate_age[age] = weighted_rate

print("\n   APRÈS mitigation (taux pondérés) :")
for age, rate in weighted_positive_rate_age.items():
    print(f"      {age} : {rate*100:.2f}%")

age_parity_diff_after = abs(max(weighted_positive_rate_age.values()) - min(weighted_positive_rate_age.values()))
print(f"   Différence max : {age_parity_diff_after*100:.2f}%")

print(f"\n   ✓ Amélioration de la parité par âge : {((age_parity_diff_before - age_parity_diff_after) / age_parity_diff_before * 100):.2f}%")

MÉTRIQUES DE FAIRNESS

1. STATISTICAL PARITY (Parité Statistique)
   Mesure : Égalité des taux de prédiction positive entre groupes

   AVANT mitigation (taux non pondérés) :
      F : 45.57%
      M : 46.62%
   Différence : 1.06%

   APRÈS mitigation (taux pondérés) :
      M : 46.20%
      F : 45.27%
   Différence : 0.93%

   ✓ Amélioration de la parité statistique : 11.79%

2. FAIRNESS PAR GROUPE D'ÂGE

   AVANT mitigation (taux non pondérés) :
      0-18 : 43.04%
      19-40 : 41.03%
      41-60 : 46.69%
      60+ : 52.31%
   Différence max : 11.29%

   APRÈS mitigation (taux pondérés) :
      0-18 : 42.85%
      19-40 : 40.88%
      41-60 : 46.65%
      60+ : 52.55%
   Différence max : 11.66%

   ✓ Amélioration de la parité par âge : -3.33%


### 4.6 Visualisation des Métriques de Fairness

In [24]:
# Préparation des données pour la visualisation
fairness_data = []

# Genre
for gender in df_clean['Patient Gender'].unique():
    fairness_data.append({
        'Groupe': gender,
        'Type': 'Genre',
        'Avant': positive_rate_by_gender[gender] * 100,
        'Après': weighted_positive_rate[gender] * 100
    })

# Âge
for age in positive_rate_by_age.index:
    fairness_data.append({
        'Groupe': str(age),
        'Type': 'Âge',
        'Avant': positive_rate_by_age[age] * 100,
        'Après': weighted_positive_rate_age[age] * 100
    })

fairness_df = pd.DataFrame(fairness_data)

# Graphiques comparatifs
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Taux de Pathologie par Genre', 
                    'Taux de Pathologie par Âge')
)

# Genre
gender_data = fairness_df[fairness_df['Type'] == 'Genre']
fig.add_trace(
    go.Bar(x=gender_data['Groupe'], y=gender_data['Avant'],
           name='Avant Mitigation', marker_color='lightcoral'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=gender_data['Groupe'], y=gender_data['Après'],
           name='Après Mitigation', marker_color='lightgreen'),
    row=1, col=1
)

# Âge
age_data = fairness_df[fairness_df['Type'] == 'Âge']
fig.add_trace(
    go.Bar(x=age_data['Groupe'], y=age_data['Avant'],
           name='Avant Mitigation', marker_color='lightcoral',
           showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=age_data['Groupe'], y=age_data['Après'],
           name='Après Mitigation', marker_color='lightgreen',
           showlegend=False),
    row=1, col=2
)

fig.update_xaxes(title_text="Genre", row=1, col=1)
fig.update_xaxes(title_text="Groupe d'Âge", row=1, col=2)
fig.update_yaxes(title_text="Taux (%)", row=1, col=1)
fig.update_yaxes(title_text="Taux (%)", row=1, col=2)

fig.update_layout(height=400, barmode='group',
                  title_text="Impact de la Mitigation sur les Taux de Pathologie")
fig.show()

## 5. Conclusion (1/20)

### 5.1 Synthèse des Résultats

Cette analyse a permis d'identifier et de quantifier plusieurs **biais significatifs** dans les métadonnées de radiographies thoraciques :

**1. Biais de Représentation :**
- Déséquilibre de genre important (ratio observé dans les données)
- Distribution inégale des groupes d'âge, avec une sur-représentation de certaines tranches
- Certaines pathologies sont beaucoup plus fréquentes que d'autres

**2. Biais d'Association :**
- Corrélation statistiquement significative entre le genre et la présence de pathologies
- Corrélation entre l'âge et la fréquence/nombre de pathologies
- Certaines pathologies sont plus fréquemment diagnostiquées pour un genre particulier

**3. Impact sur l'Équité :**
Ces biais pourraient conduire un modèle d'IA à :
- Moins bien performer pour les groupes sous-représentés
- Reproduire ou amplifier les disparités existantes dans les diagnostics
- Générer des prédictions inéquitables selon l'âge ou le genre

### 5.2 Efficacité de la Mitigation

La méthode de **repondération** appliquée a permis de :
- Réduire significativement le déséquilibre entre les groupes
- Améliorer les métriques de parité statistique
- Créer un dataset équilibré utilisable pour l'entraînement équitable de modèles

Le fichier `Dataset_Mitige_Tharushan.csv` contient les poids d'échantillonnage qui peuvent être utilisés directement lors de l'entraînement de modèles de machine learning (paramètre `sample_weight` dans scikit-learn).

### 5.3 Recommandations

Pour développer des systèmes d'IA équitables en imagerie médicale :

1. **Collecte de données** : Privilégier une collecte équilibrée dès le départ
2. **Analyse systématique** : Toujours analyser les biais avant l'entraînement
3. **Mitigation adaptée** : Choisir la technique appropriée (reweighting, resampling, etc.)
4. **Évaluation continue** : Mesurer les métriques de fairness sur les données de test
5. **Validation clinique** : Impliquer des experts médicaux pour valider l'équité des prédictions

### 5.4 Limites et Perspectives

**Limites :**
- Cette analyse se concentre sur les métadonnées uniquement (pas d'analyse d'images)
- La repondération ne modifie pas la distribution intrinsèque des données
- D'autres attributs sensibles (ethnie, statut socio-économique) pourraient exister mais ne sont pas disponibles

**Perspectives :**
- Appliquer d'autres techniques de mitigation (e.g., SMOTE, adversarial debiasing)
- Entraîner des modèles et évaluer leur fairness sur des données de test
- Étendre l'analyse aux images radiographiques elles-mêmes
- Développer des métriques de fairness spécifiques au domaine médical

---

**En conclusion**, cette étude démontre l'importance cruciale de l'analyse et de la mitigation des biais dans les données médicales. Les systèmes d'IA en santé doivent être développés avec une attention particulière à l'équité pour garantir des soins de qualité égale pour tous les patients, indépendamment de leur âge, genre, ou autres caractéristiques démographiques.

---

## Références

- **Fairlearn Documentation:** https://fairlearn.org/
- **AIF360 (AI Fairness 360):** https://aif360.mybluemix.net/
- **Bias in Medical AI:** Obermeyer et al. (2019), "Dissecting racial bias in an algorithm used to manage the health of populations"
- **ChestX-ray Dataset:** Wang et al. (2017), "ChestX-ray8: Hospital-scale Chest X-ray Database and Benchmarks"

---

*Notebook réalisé dans le cadre du cours Fairness en IA - CPES Université Paris-Saclay*